In [1]:
%matplotlib inline

# 1) Wipe out your namespace
%reset -f

# 2) Clear Jupyter’s stored outputs (and inputs if you like)
try:
    Out.clear() 
except NameError:
    pass

try:
    In.clear()
except NameError:
    pass

# 3) Force Python GC
import gc
gc.collect()

# 4) Free any GPU buffers
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()


import importlib
from libs import trades, plots, params, feats, strategies, models_core
importlib.reload(trades)
importlib.reload(plots)
importlib.reload(params)
importlib.reload(feats)
importlib.reload(strategies)
importlib.reload(models_core)

<module 'libs.models_core' from '/workspace/my_models/Trading/_Stock_Analysis_/libs/models_core.py'>

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import glob
import os
import json
import re
import os
import psutil
import math
from re import findall

import time
from pathlib import Path
import pickle
import datetime as dt
from datetime import datetime
from datetime import time

import torch.nn.functional as Funct
from torch.utils.data import Dataset, DataLoader
torch.serialization.add_safe_globals([models_core.DayWindowDataset])

# from tqdm.auto import tqdm
from tqdm import tqdm
import json
from PIL import Image
import IPython.display as disp
from IPython.display import clear_output, display

# Turn off interactive plotting globally (we’ll manage our own display)
import matplotlib
import matplotlib.pyplot as plt
plt.ioff()

import seaborn as sns
from pprint import pprint

import optuna
from optuna.trial import TrialState
from optuna.importance import get_param_importances
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.storages import RDBStorage

In [3]:
# ##############################
# ###### ML STRATEGIES DF ######
# ##############################

# df_trainval = pd.read_csv(params.trainval_csv, index_col=0, parse_dates=True)
# baseline_pnl = round((df_trainval["bid"].iloc[-1] - df_trainval["ask"].iloc[0]) / df_trainval.index.normalize().nunique(),4)
# print('baseline profit & loss per day:', baseline_pnl)
# df_trainval

In [4]:
##############################
## INDICATORS STRATEGIES DF ##
##############################

# Read sign timestamps ( since selected year, with pre-market included)
df_inds_trainval = pd.read_csv(params.indunsc_trainval_csv, index_col=0, parse_dates=True)
# compute synthetic ask/bid from close_raw and configured spread
df_inds_trainval['ask'] = df_inds_trainval['close'] * (1 + params.bidask_spread_pct/100)
df_inds_trainval['bid'] = df_inds_trainval['close'] * (1 - params.bidask_spread_pct/100)

df_inds_trainval

,open,high,low,close,volume,signal,ret,log_ret,sma_9,sma_21,sma_pct_9,sma_pct_21,ema_9,ema_21,roc_9,roc_21,body,body_pct,upper_shad,lower_shad,range_pct,rsi_6,macd_line,macd_signal,macd_diff,atr_14,atr_pct_14,bb_lband_20,bb_hband_20,bb_w_20,plus_di,minus_di,adx,obv,vwap_14,vwap_dev_pct_14,vol_spike,vol_z_14,rolling_max_close_21,rolling_min_close_21,dist_high_21,dist_low_21,obv_sma,obv_diff_14,obv_pct_14,obv_sma_14,obv_z_14,ret_30,sma_30,sma_pct_30,ema_30,roc_30,rsi_30,atr_30,atr_pct_30,bb_lband_30,bb_hband_30,bb_w_30,plus_di_30,minus_di_30,adx_30,obv_diff_30,obv_pct_30,obv_sma_30,obv_z_30,vwap_30,vwap_dev_pct_30,z_vwap_dev_30,vol_spike_30,vol_z_30,ret_std_30,rolling_max_close_30,rolling_min_close_30,dist_high_30,dist_low_30,macd_line_30,macd_signal_30,macd_diff_30,ret_45,sma_45,sma_pct_45,ema_45,roc_45,rsi_45,atr_45,atr_pct_45,bb_lband_45,bb_hband_45,bb_w_45,plus_di_45,minus_di_45,adx_45,obv_diff_45,obv_pct_45,obv_sma_45,obv_z_45,vwap_45,vwap_dev_pct_45,z_vwap_dev_45,vol_spike_45,vol_z_45,ret_std_45,rolling_max_close_45,rolling_min_close_45,dist_high_45,dist_low_45,macd_line_45,macd_signal_45,macd_diff_45,ret_60,sma_60,sma_pct_60,ema_60,roc_60,rsi_60,atr_60,atr_pct_60,bb_lband_60,bb_hband_60,bb_w_60,plus_di_60,minus_di_60,adx_60,obv_diff_60,obv_pct_60,obv_sma_60,obv_z_60,vwap_60,vwap_dev_pct_60,z_vwap_dev_60,vol_spike_60,vol_z_60,ret_std_60,rolling_max_close_60,rolling_min_close_60,dist_high_60,dist_low_60,macd_line_60,macd_signal_60,macd_diff_60,volume_z_30,vol_spike_z_30,ret_std_z_30,obv_pct_z_30,vwap_dev_pct_z_30,bb_w_z_30,atr_z_30,macd_diff_z_30,volume_z_45,vol_spike_z_45,ret_std_z_45,obv_pct_z_45,vwap_dev_pct_z_45,bb_w_z_45,atr_z_45,macd_diff_z_45,volume_z_60,vol_spike_z_60,ret_std_z_60,obv_pct_z_60,vwap_dev_pct_z_60,bb_w_z_60,atr_z_60,macd_diff_z_60,minute_time,hour_time,dow_time,month_time,day_of_year_time,week_of_year_time,in_sess_time,eng_ma,eng_macd,eng_bb,eng_bb_mid,eng_rsi,eng_adx,eng_obv,eng_atr_div,z_eng_atr,eng_sma_short,eng_sma_long,eng_vwap,z_vwap_dev,z_bb_w,z_obv,mom_sum_30,mom_std_30,mom_sum_45,mom_std_45,mom_sum_60,mom_std_60,eng_ema_cross_up,eng_ema_cross_down,ask,bid
2009-01-02 10:58:00,3.069643,3.069643,3.069643,3.069643,13160.0,1.645129e-12,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.0000,0.000000,0.00,0.0000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,1566040.0,3.069643,0.000000,1.000000,0.000000,3.069643,3.069643,0.000000,0.000000,1.480500e+06,184240.0,60020.011635,1.480500e+06,1.553797,0.000000,3.069643,0.000000,3.069643,0.000000,100.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,394800.0,128614.310646,1.375220e+06,0.674491,3.069643,-1.446713e-14,0.000000,1.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.069643,0.000000,3.069643,0.000000,100.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,592200.0,192921.465969,1.276520e+06,0.674491,3.069643,-1.446713e-14,0.000000,1.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.069643,0.000000,3.069643,0.000000,100.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,789600.0,257228.621291,1.177820e+06,0.674491,3.069643,-1.446713e-14,0.000000,1.000000,0.000000,0.000000,3.069643,3.069643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.352778,0.312500,0.071429,0.500000,0.502740,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.30000,0.000000,0.127854,0.000000,0.000000,0.000000,0.000000,-100.000000,0.000000,0.000000,1.388730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.070257,3.069029
2009-01-02 10:59:00,3.069643,3.069643,3.069643,3.0

In [5]:
df = df_inds_trainval # SELECT THE CORRECT DATAFRAME

baseline_pnl = round((df["bid"].iloc[-1] - df["ask"].iloc[0]) / df.index.normalize().nunique(),4)
baseline_pnl

0.0367

In [13]:
# Corrected trading_combinations using only columns produced by standard_indicators (defaults)
trading_combinations = {
    "tc_001": {"col_signal":"ema_9",               "sign_thresh":"ema_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_002": {"col_signal":"ema_9",               "sign_thresh":"ema_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_003": {"col_signal":"sma_9",               "sign_thresh":"sma_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_004": {"col_signal":"sma_9",               "sign_thresh":"sma_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_005": {"col_signal":"close",               "sign_thresh":"ema_9",             "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_006": {"col_signal":"close",               "sign_thresh":"ema_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_007": {"col_signal":"macd_line",            "sign_thresh":"macd_signal",       "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_008": {"col_signal":"macd_line",            "sign_thresh":"macd_signal",       "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_009": {"col_signal":"macd_diff",            "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_010": {"col_signal":"macd_diff",            "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_011": {"col_signal":"dist_low_21",          "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_012": {"col_signal":"dist_low_21",          "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_013": {"col_signal":"dist_high_21",         "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_014": {"col_signal":"dist_high_21",         "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_015": {"col_signal":"sma_pct_9",            "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_016": {"col_signal":"sma_pct_9",            "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_017": {"col_signal":"roc_9",                "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_018": {"col_signal":"roc_9",                "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_019": {"col_signal":"vwap_dev_pct_14",      "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_020": {"col_signal":"vwap_dev_pct_14",      "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_021": {"col_signal":"obv_pct_14",           "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_022": {"col_signal":"obv_pct_14",           "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_023": {"col_signal":"vol_spike",            "sign_thresh":1.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_024": {"col_signal":"vol_spike",            "sign_thresh":1.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_025": {"col_signal":"rolling_max_close_21", "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_026": {"col_signal":"rolling_min_close_21", "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_027": {"col_signal":"sma_9",                "sign_thresh":"ema_9",             "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_028": {"col_signal":"ema_9",                "sign_thresh":"sma_9",             "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_029": {"col_signal":"sma_pct_9",            "sign_thresh":"sma_pct_21",        "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_030": {"col_signal":"sma_pct_9",            "sign_thresh":"sma_pct_21",        "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_031": {"col_signal":"obv_sma",              "sign_thresh":"obv_sma_14",        "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_032": {"col_signal":"obv_sma",              "sign_thresh":"obv_sma_14",        "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_033": {"col_signal":"obv_diff_14",          "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_034": {"col_signal":"obv_diff_14",          "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_035": {"col_signal":"obv_z_14",             "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_036": {"col_signal":"obv_z_14",             "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_037": {"col_signal":"body_pct",             "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_038": {"col_signal":"upper_shad",           "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_039": {"col_signal":"plus_di",              "sign_thresh":"minus_di",          "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_040": {"col_signal":"adx",                  "sign_thresh":20.0,                "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_041": {"col_signal":"bb_w_20",              "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_042": {"col_signal":"bb_w_20",              "sign_thresh":0.0,                 "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_043": {"col_signal":"vwap_dev_pct_14",      "sign_thresh":"sma_pct_9",         "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_044": {"col_signal":"vwap_dev_pct_14",      "sign_thresh":"ema_9",             "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_045": {"col_signal":"roc_9",                "sign_thresh":"ema_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_046": {"col_signal":"roc_9",                "sign_thresh":"ema_21",            "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},

    "tc_047": {"col_signal":"rolling_min_close_21", "sign_thresh":"dist_low_21",       "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":"vwap_14"},
    "tc_048": {"col_signal":"rolling_max_close_21", "sign_thresh":"dist_high_21",      "col_atr":"atr_14", "col_rsi":"rsi_6", "col_vwap":None},
}


In [14]:
# Pre-compute per-day slices once
groups = df.groupby(df.index.normalize())
prun_perc = 80 # higher value means the pruner keeps more trials (e.g., 90 keeps the top 10%)
report_interval = 50
_findall = re.compile(r"[+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?").findall

def objective(trial: optuna.Trial) -> float:
    
    sellmin_idx    = trial.suggest_categorical("sellmin_idx", [None, -1, -2, -3])
    trailstop_pct = trial.suggest_float("trailstop_pct", 0.01, 0.30, log=True)
    rsi_thresh    = trial.suggest_int("rsi_thresh", 30, 75)
    atr_mult      = trial.suggest_float("atr_mult", 0.1, 6.0, log=True)
    vwap_atr_mult = trial.suggest_float("vwap_atr_mult", -4.0, 4.0)
    trad_comb      = trading_combinations[trial.suggest_categorical("tc_id", list(trading_combinations.keys()))]
    sess_start     = time.fromisoformat(trial.suggest_categorical("sess_start", [params.sess_start_pred.isoformat(), params.sess_start_reg.isoformat()]))

    print('-' * 150)
    print(f"Params => sellmin_idx={sellmin_idx}, trailstop_pct={trailstop_pct:.4f}, rsi_thresh={rsi_thresh:.2f}, atr_mult={atr_mult:.3f}, vwap_atr_mult={vwap_atr_mult:.3f}, sess_start={sess_start}")
    
    daily_pnls: List[float] = []
    pbar = tqdm(groups, desc=f"Trial {trial.number}", leave=True)
    for step, (day, df_day) in enumerate(pbar, 1):

        df_actions = strategies.generate_tradact_elab(
            df              = df_day,
            col_signal      = trad_comb["col_signal"],
            sign_thresh     = trad_comb["sign_thresh"],
            col_atr         = trad_comb["col_atr"],
            col_rsi         = trad_comb["col_rsi"],
            col_vwap        = trad_comb["col_vwap"],
            sellmin_idx     = sellmin_idx,
            trailstop_pct   = trailstop_pct,
            rsi_thresh      = rsi_thresh,
            atr_mult        = atr_mult,
            vwap_atr_mult   = vwap_atr_mult,
            sess_start      = sess_start
        )

        sim = strategies.simulate_trading(
            day               = day,
            df                = df_actions,
            sellmin_idx       = sellmin_idx, 
            sess_start        = sess_start
        )
   
        _, _, stats = next(iter(sim.values()))
        
        strategy_val = float(_findall(stats["STRATEGY"])[-1]) 

        daily_pnls.append(strategy_val)

        # Explicitly delete large locals at the end of each trial (no per-iteration gc.collect)
        del df_actions, sim, stats
        df_day = df_actions = sim = stats = None

        # Rreport for pruning
        if step % report_interval == 0:
            report_step = step // report_interval
            prun_val = float(np.mean(daily_pnls))
            trial.report(prun_val, report_step)
            if trial.should_prune():
                vals = [t.intermediate_values[report_step]
                        for t in trial.study.get_trials(deepcopy=False)
                        if t.state == TrialState.COMPLETE and report_step in t.intermediate_values]
                thr = float(__import__("numpy").percentile(vals, prun_perc))
                print(f"[PRUNE] value={prun_val:.6f} thr={thr:.6f}")
                pbar.close()
                raise optuna.TrialPruned()

    # Average daily P&L
    return float(np.mean(daily_pnls))


In [15]:
# hide Optuna's default INFO messages
optuna.logging.set_verbosity(optuna.logging.ERROR)

n_trials = 3000
n_jobs = 1

reports_per_trial = math.ceil(len(groups) / report_interval) # at least 30% of days to execute before pruning
n_warmup_steps = math.ceil(reports_per_trial * 0.30)


def short_log_callback(study, trial):
    val = trial.value
    best_val = study.best_value
    best_idx = study.best_trial.number if study.best_trial is not None else None
    pct_improv = ((float(val) - baseline_pnl) / baseline_pnl) * 100
    print(f"[Results] Trial {trial.number} finished with value: {float(val):.4f}. Improvement over baseline: {pct_improv:.2f}%. \
    Best is trial {best_idx} with value: {float(best_val):.4f}.")


pruner = optuna.pruners.PercentilePruner(percentile=prun_perc, n_startup_trials=30, n_warmup_steps=n_warmup_steps)

study = optuna.create_study(
    storage=None,
    load_if_exists=True,
    direction="maximize",
    pruner=pruner,
)

study.optimize(
    objective,
    n_trials          = n_trials,
    n_jobs            = n_jobs,
    callbacks         = [plots.plot_callback, plots.save_results_callback, short_log_callback],
    gc_after_trial    = True,
)

plt.close('all')   # safe here; the final image remains displayed in the notebook output
gc.collect()       # optional extra sweep

------------------------------------------------------------------------------------------------------------------------------------------------------
Params => sellmin_idx=None, trailstop_pct=0.0234, rsi_thresh=50.00, atr_mult=0.187, vwap_atr_mult=-3.359, sess_start=14:30:00


Trial 0:   0%|          | 0/3519 [00:02<?, ?it/s]


KeyError: None

In [ ]:
# Final plots & JSON dump 
ax = plot_optimization_history(study)
ax.figure.set_size_inches(8, 4)
plt.show()

print("Best Parameters       :", study.best_params)
print("Best Average Daily P&L:", study.best_value)

importances = get_param_importances(study)
print("\nHyperparameter importances (higher ⇒ more impact):")
for name, score in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"  {name:20s}: {score:.3f}")

first_day = df.index.normalize().min().strftime("%Y%m%d")
last_day  = df.index.normalize().max().strftime("%Y%m%d")
file_name = f"{params.ticker}_{round(study.best_value,4)}_predicted.json"
file_path = os.path.join(params.optuna_folder, file_name)

with open(file_path, "w") as f:
    json.dump({
        "best_params": study.best_params,
        "best_value" : study.best_value,
        "importances": importances,
        "trials": [
            {"number": t.number, "value": t.value, "params": t.params, "state": t.state.name}
            for t in study.trials
        ],
    }, f, indent=4)

print(f"\nOptuna results (and importances) saved to: {file_path}")